![msg](msg.jpg)

In [ ]:
#Imports 
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 

from tqdm import tqdm

from DiscEvolution.grid import Grid
from DiscEvolution.star import SimpleStar
from DiscEvolution.eos import LocallyIsothermalEOS  
from DiscEvolution.disc import AccretionDisc
from DiscEvolution.viscous_evolution import ViscousEvolutionFV
from DiscEvolution.driver import PlanetDiscDriver
from DiscEvolution.io import Event_Controller

In [ ]:
# Physical parameters
#Assumed
Mstar = 1.0 * u.solMass #Default 1 solmass 
Rstar = 1.0 * u.solRad #Default 1 solRad
mu = 2.33  # Mean molecular weight, assumed

#Given
M_disk = 0.05 * u.solMass  #Initial Disk Mass, default is 0.05 solmass
gamma = 1.0
Rc = 1.0 * u.AU #Default 1AU
#Mdot_0 = 
t_final = 1 * u.Myr 

# LBP surface density profile with gamma = 1: Sigma(r) = (M_disk / (2*pi*Rc*r)) * exp(-r/Rc)
def Sigma_LBP(r) -> float:
    val = (M_disk)/(2* np.pi * Rc * r) * np.exp((-1*r)/(Rc))
    if (val.cgs.unit != (u.g / (u.cm)**2)):
        print("Sigma_1 error")
        return 0
    return val.cgs

def Temperature(r) -> float:
    T_0 = 150 * u.K 
    val  = T_0 * (r / (1.0 * u.AU))**(-1/2)
    if val.cgs.unit != u.K:
        print('Temperature error')
        return 0
    return val.cgs

#Taken from A2
def Omega(r) -> float:
    val = np.sqrt((const.G*Mstar)/(r**3))
    if (val.cgs.unit != (1 / u.s)): 
        print("Omega error")
        return 0
    return val.cgs

#Taken from A2
def c_s(r) -> float:
    val = np.sqrt((const.k_B*Temperature(r))/(mu*const.m_p))
    if (val.cgs.unit != (u.cm / u.s)): 
        print("c_s error")
        return 0
    return val.cgs

def H(r) -> float: 
    val = c_s(r)/Omega(r)
    if val.cgs.unit != (u.cm):
        print("H error")
        return 0
    return val.cgs

In [ ]:
# TEST: Use STANDARD LocallyIsothermalEOS (T ∝ r^(-3/4)) to verify framework works
# Then we'll understand what's different with LBP temperature

# Grid parameters
R_in = 0.1  # AU
R_out = 30.0  # AU
Ncells = 100

# Create grid and star
grid = Grid(R_in, R_out, Ncells, spacing='log')
star = SimpleStar(M=1.0)  # Unitless solar masses

# Standard LocallyIsothermalEOS with T ∝ r^(-3/4) (the default)
# h0 = H/R at 1 AU, typically ~0.033
# cs0 = sound speed at 1 AU
T_1AU = 150  # K at 1 AU
cs_1AU = np.sqrt(const.k_B.cgs.value * T_1AU / (mu * const.m_p.cgs.value))  # cm/s
h_1AU = cs_1AU / np.sqrt(const.G.cgs.value * (1.0 * u.solMass).to(u.g).value / (1.0 * u.AU).to(u.cm).value**3)
h0_ratio = h_1AU / (1.0 * u.AU).to(u.cm).value  # H/R at 1 AU

print(f"=== STANDARD EOS SETUP ===")
print(f"T at 1 AU: {T_1AU} K")
print(f"cs at 1 AU: {cs_1AU:.2e} cm/s")
print(f"H at 1 AU: {h_1AU:.2e} cm")
print(f"h0 (H/R at 1 AU): {h0_ratio:.4f}")

alpha = 0.01
eos = LocallyIsothermalEOS(star, h0=h0_ratio, q=-0.25, alpha_t=alpha)  # q=-0.25 gives T ∝ r^(-1/2)
eos.set_grid(grid)

# LBP surface density
Sigma_init = np.array([Sigma_LBP_1(r * u.AU).value for r in grid.Rc])
disc = AccretionDisc(grid, star, eos, Sigma=Sigma_init)

print(f"\n=== DISC CHECK ===")
print(f"Disc mass: {disc.Mtot() * u.g.to(u.solMass):.3e} M_sun")

idx_1AU = np.argmin(np.abs(disc.R - 1.0))
print(f"\n=== AT 1 AU ===")
print(f"T: {disc.T[idx_1AU]:.1f} K")
print(f"cs: {disc.cs[idx_1AU]:.2e} cm/s")
print(f"H: {disc.H[idx_1AU]:.2e} cm")
print(f"nu: {disc.nu[idx_1AU]:.2e} cm²/s")
print(f"Expected nu = α*cs*H = {alpha * disc.cs[idx_1AU] * disc.H[idx_1AU]:.2e} cm²/s")

print(f"\n=== VISCOSITY RANGE ===")
print(f"nu min: {disc.nu.min():.2e} cm²/s at R = {disc.R[np.argmin(disc.nu)]:.2f} AU")
print(f"nu max: {disc.nu.max():.2e} cm²/s at R = {disc.R[np.argmax(disc.nu)]:.2f} AU")

# Check timestep
visc_evol = ViscousEvolutionFV(tol=0.5, boundary='power_law', in_bound='Mdot')
dt_max = visc_evol.max_timestep(disc)
print(f"\n=== TIMESTEP ===")
print(f"dt_max: {dt_max:.2e} seconds")
print(f"dt_max: {dt_max * (1*u.s).to(u.yr).value:.2e} years")
print(f"dt_max: {dt_max * (1*u.s).to(u.Myr).value:.2e} Myr")

t_final_sec = (1 * u.Myr).to(u.s).value
n_steps = t_final_sec / dt_max
print(f"Estimated steps for 1 Myr: {n_steps:.2e}")